# Overview

In [ ]:
text = 'the quick for jumped over a lazy dog'
# 1. tokenize
# -> ['the','quick',,,]

# 2. vocab
# -> ['the' -> 0,
#    'quick' -> 1,
#    ...

# 3. numericalize
# text -> idx

# 4. embedding
# idx -> latent vector

# Process
1. specify how preprocessing should be done => Fields
2. Use Dataset to load the data => TabularDataset (Json/csv/tsv)
3. Construct an iterator to do batching & padding => BucketIterator

In [4]:
from torchtext.data import Field, TabularDataset, BucketIterator
import spacy

# ---------0. tokenizer ----------------------------
# tokenize = lambda x: x.split()
spacy_en = spacy.load("en") # python -m spacy download en

def tokenize(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

# --------- 1. how the text should be tokenized ----------
quote = Field(sequential = True,
              use_vocab = True, # if false data should be numerical
              tokenize = tokenize,
              lower = True)
    # 정의한 토크나이저로 전처리 하는 메서드
score = Field(sequential = False, # 시계열이 아님
              use_vocab = False)
    # target : sentiment analysis

# -------------------- this goes to TabularDataset ------------------------
fields = {"quote": ('q', quote), 'score':('s',score)}
    # 데이터 셋에 있는 어떤 필드를 사용하고 싶나(?)
    
# ---------------------------------------------------------
train_data, test_data = TabularDataset.splits(
                                    path = 'mydata',
                                    train = 'train.json',
                                    test = 'test.json',
                                    #validation = 'validation.json'
                                    format = 'json',
                                    fields = fields)
    # 데이터를 불러와서, fields를 통해 전처리를 하고 이를 train, test에 보낸다.

# print(train_data[0].__dict__.keys())
# print(train_data[0].__dict__.values())


quote.build_vocab(train_data,
                  max_size = 10000,
                  min_freq = 2, # 2번이상은 나와야 워드로 인식
                  #vectors = 'glove.6B.100d',
                 )

# 만약 머신 트랜스레이터라면 score도 build_vocab해줘야겟지? 


train_iterator, test_iterator = BucketIterator.splits(
                                    (train_data, test_data),
                                    batch_size = 2,
                                    device = 'cuda')



for batch in train_iterator:
    print(batch.q) # train_iterator에 들어있는 X 텐서들이 나옴. padding은 1로 부여되었음.
    print(batch.s) # train_iterator에 있는 Y텐서들이 나옴.
    
    

FileNotFoundError: [Errno 2] No such file or directory: 'mydata/train.json'